In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2025-11-24 14:04:15.357067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763973255.429500   80074 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763973255.448940   80074 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763973255.613077   80074 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763973255.613106   80074 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763973255.613108   80074 computation_placer.cc:177] computation placer alr

In [2]:
df = pd.read_csv('diabetes.csv')
print(df.head())

   Glucose  Insulin   BMI  Age  Outcome
0      148        0  33.6   50        1
1       85        0  26.6   31        0
2      183        0  23.3   32        1
3       89       94  28.1   21        0
4      137      168  43.1   33        1


In [3]:
df.describe()

,Glucose,Insulin,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000
mean,120.894531,79.799479,31.992578,33.240885,0.348958
std,31.972618,115.244002,7.884160,11.760232,0.476951
min,0.000000,0.000000,0.000000,21.000000,0.000000
25%,99.000000,0.000000,27.300000,24.000000,0.000000
50%,117.000000,30.500000,32.000000,29.000000,0.000000
75%,140.250000,127.250000,36.600000,41.000000,1.000000
max,199.000000,846.000000,67.100000,81.000000,1.000000


In [4]:
X = df.drop(labels='Outcome',axis=1)
y = df['Outcome']

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Gaussian Naive Bayesian

In [6]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

model.fit(X_train_scaled,y_train)

y_pred_GNB = model.predict(X_test_scaled)
print(y_pred_GNB[:10])

[0 0 0 0 0 1 0 0 1 1]


MLP

In [ ]:
class Perceptron:
    def __init__(self,learning_rate = 0.01, n_iterations = 1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
    def fit(self, X, y):
        n_samples ,n_features = X.shape
        
        self.weights = np.zeros(n_features)
        self.bias = 0
               
        y_ = np.where(y <= 0, -1, 1)


        for _ in range(self.n_iterations):
            for idx, x_i in enumerate(X):
                
                linear_output = np.dot(x_i, self.weights) + self.bias 

                y_predicted = np.sign(linear_output)

                if y_predicted != y_[idx]:
                    update = self.learning_rate * y_[idx]
                    self.weights += update * x_i
                    self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = np.sign(linear_output)
        
        return np.where(y_predicted == -1, 0, 1)

In [8]:
custom_mlp = Perceptron(learning_rate=0.01,n_iterations=1000)
custom_mlp.fit(X_train_scaled,y_train)

y_pred_mlp = custom_mlp.predict(X_test_scaled)
print(y_pred_mlp[:10])

[0 0 0 0 1 1 0 1 1 1]


In [9]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

accuracy_GNB = accuracy_score(y_test,y_pred_GNB)
precision_GNB = precision_score(y_test,y_pred_GNB)
recall_GNB = recall_score(y_test,y_pred_GNB)
f1_GNB = f1_score(y_test,y_pred_GNB)

print("Metrics from GNB:")
print(f'Accuracy_GNB: {accuracy_GNB}')
print(f'Precision_GNB: {precision_GNB}')
print(f'Recall_GNB: {recall_GNB}')
print(f'F1_GNB: {f1_GNB}')

print("\n")
print("Metrics from MLP:")

accuracy_MLP = accuracy_score(y_test,y_pred_mlp)
precision_MLP = precision_score(y_test,y_pred_mlp)
recall_MLP = recall_score(y_test,y_pred_mlp)
f1_MLP = f1_score(y_test,y_pred_mlp)

print(f'Accuracy_MLP: {accuracy_MLP}')
print(f'Precision_MLP: {precision_MLP}')
print(f'Recall_MLP: {recall_MLP}')
print(f'F1_MLP: {f1_MLP}')

Metrics from GNB:
Accuracy_GNB: 0.7467532467532467
Precision_GNB: 0.6481481481481481
Recall_GNB: 0.6363636363636364
F1_GNB: 0.6422018348623854


Metrics from MLP:
Accuracy_MLP: 0.6753246753246753
Precision_MLP: 0.5301204819277109
Recall_MLP: 0.8
F1_MLP: 0.6376811594202898


1. Accuracy: Gaussian Naïve Bayes (74.7%) is significantly more accurate than the MLP (67.5%).

Reason: The diabetes dataset is not "linearly separable". The MLP is trying to force a straight line through complex data, whereas GNB uses probability curves (Gaussian distribution), allowing it to capture the "shape" of the data better.

2. Precision vs. Recall Trade-off
This is the most interesting part of your results.

Precision (GNB is cleaner): GNB (64.8%) vs MLP (53.0%).

Reason: When the MLP predicts someone has diabetes, it is wrong nearly half the time (low precision). It generates a lot of "False Positives." GNB is much more trustworthy when it claims a patient is sick.

Recall (MLP is better): MLP (80.0%) vs GNB (63.6%).

Interpretation: MLP is actually better at catching sick people. It identified 80% of the diabetic patients in the test set, whereas GNB missed nearly 40% of them.

Which is more significant, Recall or Precision: In medical screening, Recall is often prioritized over Precision. You would rather falsely warn a healthy person (False Positive) than tell a sick person they are fine (False Negative).

3. F1-Score: The scores are almost identical (~64%).

Interpretation: This shows that while the models behave very differently, their "overall" effectiveness is similar. GNB is balanced; MLP sacrifices precision to get high recall.

In [10]:
import pickle

In [11]:
with open('naive_bayes_model.pkl', 'wb') as file:
    pickle.dump(model, file)

with open('mlp_model.pkl', 'wb') as file:
    pickle.dump(custom_mlp, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("Models saved successfully!")


Models saved successfully!


In [12]:
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("Gaussian Naive Bayes 5-Fold CV:")

gnb_scores = cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring='accuracy')

for i, score in enumerate(gnb_scores, 1):
    print(f"--- Fold {i}/5 ---")
    print(f"Accuracy for this fold: {score*100:.2f}%\n")

print("--- K-Fold Cross-Validation Results ---")
print(f"Average Accuracy: {gnb_scores.mean()*100:.2f}%")
print(f"Standard Deviation: {gnb_scores.std()*100:.2f}%")
print("\n" + "="*40 + "\n")

print("Custom MLP 5-Fold CV:")

mlp_accuracies = []
y_train_np = y_train.to_numpy()

fold_num = 1
for train_index, val_index in kf.split(X_train_scaled):
    X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
    y_fold_train, y_fold_val = y_train_np[train_index], y_train_np[val_index]
    
    fold_model = Perceptron(learning_rate=0.01, n_iterations=1000)
    fold_model.fit(X_fold_train, y_fold_train)
    
    predictions = fold_model.predict(X_fold_val)
    accuracy = np.mean(predictions == y_fold_val)
    mlp_accuracies.append(accuracy)
    
    print(f"--- Fold {fold_num}/5 ---")
    print(f"Accuracy for this fold: {accuracy*100:.2f}%\n")
    fold_num += 1

mlp_accuracies = np.array(mlp_accuracies)

print("--- K-Fold Cross-Validation Results ---")
print(f"Average Accuracy: {mlp_accuracies.mean()*100:.2f}%")
print(f"Standard Deviation: {mlp_accuracies.std()*100:.2f}%")

Gaussian Naive Bayes 5-Fold CV:
--- Fold 1/5 ---
Accuracy for this fold: 77.24%

--- Fold 2/5 ---
Accuracy for this fold: 75.61%

--- Fold 3/5 ---
Accuracy for this fold: 73.17%

--- Fold 4/5 ---
Accuracy for this fold: 72.36%

--- Fold 5/5 ---
Accuracy for this fold: 74.59%

--- K-Fold Cross-Validation Results ---
Average Accuracy: 74.59%
Standard Deviation: 1.73%


Custom MLP 5-Fold CV:


--- Fold 1/5 ---
Accuracy for this fold: 57.72%

--- Fold 2/5 ---
Accuracy for this fold: 73.98%

--- Fold 3/5 ---
Accuracy for this fold: 71.54%

--- Fold 4/5 ---
Accuracy for this fold: 71.54%

--- Fold 5/5 ---
Accuracy for this fold: 76.23%

--- K-Fold Cross-Validation Results ---
Average Accuracy: 70.21%
Standard Deviation: 6.48%
